# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns # Matplotlib 的進階視覺庫
import os
from sklearn.model_selection import train_test_split # 切分資料

data_path = './data'

In [2]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [3]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
#df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
#df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [4]:
char_to_id = {}
id_to_char = {}

# 建立字典並為訓練數據集中的每個標記分配ID
# 字典應包含 <eos> 和 <pad>
# char_to_id 用於將字符轉換為ID，而 id_to_char 則相反
special_tokens = ['<pad>', '<eos>']

# 包含 src 和 tgt 中的所有字符
all_chars = set(''.join(df_train['src']) + ''.join(df_train['tgt']) + ''.join(df_eval['src']) + ''.join(df_eval['tgt']))

# 分離數字和符號（可選，根據需要）
digits = sorted([char for char in all_chars if char.isdigit()])
symbols = sorted([char for char in all_chars if not char.isdigit()])

# 分配特殊標記的 ID
char_to_id = {'<pad>': 0, '<eos>': 1}

# 分配其他字符的 ID
char_to_id.update({char: idx + 2 for idx, char in enumerate(digits + symbols)})
id_to_char = {idx: char for char, idx in char_to_id.items()}

vocab_size = len(char_to_id)
print('Vocab size: {}'.format(vocab_size))

Vocab size: 18


In [5]:
# 檢查數據表
char_to_id_df = pd.DataFrame(list(char_to_id.items()), columns=['ID', 'Character'])
print(char_to_id_df)

       ID  Character
0   <pad>          0
1   <eos>          1
2       0          2
3       1          3
4       2          4
5       3          5
6       4          6
7       5          7
8       6          8
9       7          9
10      8         10
11      9         11
12      (         12
13      )         13
14      *         14
15      +         15
16      -         16
17      =         17


# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [6]:
df_train = df_train[['src', 'tgt', 'len']]
df_train['char_id_list'] = df_train['src'].apply(lambda x: [char_to_id[char] for char in x] + [char_to_id['<eos>']])
df_train['label_id_list'] = df_train['tgt'].apply(lambda y: [char_to_id['<pad>']] * (len(df_train['src'].iloc[0]) - len(y)) + [char_to_id[char] for char in y] + [char_to_id['<eos>']])

df_train.head()

,src,tgt,len,char_id_list,label_id_list
0,14*(43+20)=,882,11,"[3, 6, 14, 12, 6, 5, 15, 4, 2, 13, 17, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 4, 1]"
1,(6+1)*5=,35,8,"[12, 8, 15, 3, 13, 14, 7, 17, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 7, 1]"
2,13+32+29=,74,9,"[3, 5, 15, 5, 4, 15, 4, 11, 17, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 6, 1]"
3,31*(3-11)=,-248,10,"[5, 3, 14, 12, 5, 16, 3, 3, 13, 17, 1]","[0, 0, 0, 0, 0, 0, 0, 16, 4, 6, 10, 1]"
4,24*49+1=,1177,8,"[4, 6, 14, 6, 11, 15, 3, 17, 1]","[0, 0, 0, 0, 0, 0, 0, 3, 3, 9, 9, 1]"


In [7]:
df_eval = df_eval[['src', 'tgt', 'len']]
df_eval['char_id_list'] = df_eval['src'].apply(lambda x: [char_to_id[char] for char in x] + [char_to_id['<eos>']])
df_eval['label_id_list'] = df_eval['tgt'].apply(lambda y: [char_to_id['<pad>']] * (len(df_eval['src'].iloc[0]) - len(y)) + [char_to_id[char] for char in y] + [char_to_id['<eos>']])
df_eval.head()

,src,tgt,len,char_id_list,label_id_list
0,48+43+34=,125,9,"[6, 10, 15, 6, 5, 15, 5, 6, 17, 1]","[0, 0, 0, 0, 0, 0, 3, 4, 7, 1]"
1,30-(48+13)=,-31,11,"[5, 2, 16, 12, 6, 10, 15, 3, 5, 13, 17, 1]","[0, 0, 0, 0, 0, 0, 16, 5, 3, 1]"
2,(21*31)+10=,661,11,"[12, 4, 3, 14, 5, 3, 13, 15, 3, 2, 17, 1]","[0, 0, 0, 0, 0, 0, 8, 8, 3, 1]"
3,2-27-10=,-35,8,"[4, 16, 4, 9, 16, 3, 2, 17, 1]","[0, 0, 0, 0, 0, 0, 16, 5, 7, 1]"
4,(15*20)+24=,324,11,"[12, 3, 7, 14, 4, 2, 13, 15, 4, 6, 17, 1]","[0, 0, 0, 0, 0, 0, 5, 4, 6, 1]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [53]:
batch_size = 64
epochs = 10
embed_dim = 256
hidden_dim = 256
lr = 0.0004
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [54]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences[index][0]
        y = self.sequences[index][1]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [55]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']].values.tolist())
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']].values.tolist())

In [57]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
dl_eval = torch.utils.data.DataLoader(ds_eval, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [58]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.dropout = torch.nn.Dropout(p=0.3)  # Dropout layer

        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.LeakyReLU(),  # LeakyReLU for more stability
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        # 將字符轉換為嵌入層，這部分保持在 GPU 上
        batch_x = self.embedding(batch_x)
        
        # 只將 batch_x_lens 移到 CPU 上，其餘保持在 GPU 上
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                            batch_x_lens,  # 確保 lengths 在 CPU 上
                                                            batch_first=True,
                                                            enforce_sorted=False)
        
        # 通過第一層 LSTM，這部分保持在 GPU 上
        packed_output, _ = self.rnn_layer1(packed_input)
        
        # 通過第二層 LSTM，這部分保持在 GPU 上
        packed_output, _ = self.rnn_layer2(packed_output)
        
        # 將序列轉換回填充的形式
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_output,
                                                            batch_first=True)
        
        # 通過線性層
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    # 修正的生成器方法
    def generator(self, start_char, max_len=50):
        # 初始序列
        char_list = [char_to_id[c] for c in start_char]
        input_tensor = torch.tensor([char_list], dtype=torch.long).to(next(self.parameters()).device)
        
        # 初始化隱藏狀態，初始化為零
        hidden_state1 = (torch.zeros(1, 1, self.rnn_layer1.hidden_size).to(next(self.parameters()).device),
                         torch.zeros(1, 1, self.rnn_layer1.hidden_size).to(next(self.parameters()).device))
        hidden_state2 = (torch.zeros(1, 1, self.rnn_layer2.hidden_size).to(next(self.parameters()).device),
                         torch.zeros(1, 1, self.rnn_layer2.hidden_size).to(next(self.parameters()).device))
        
        # 逐步生成字符
        while len(char_list) < max_len: 
            # 將 char_list 轉換為 tensor 並嵌入
            input_tensor = torch.tensor([[char_list[-1]]], dtype=torch.long).to(next(self.parameters()).device)
            embedded = self.embedding(input_tensor)

            # 前向傳播到 LSTM 層
            output, hidden_state1 = self.rnn_layer1(embedded, hidden_state1)
            output, hidden_state2 = self.rnn_layer2(output, hidden_state2)
            
            # 通過線性層獲取預測
            output = self.linear(output)
            y = output[:, -1, :]
            
            # 獲取概率最大的字符
            next_char = torch.argmax(y, dim=1).item()
            
            # 如果是 <eos>，則停止生成
            if next_char == char_to_id['<eos>']:
                break
            
            # 否則，添加到 char_list 中
            char_list.append(next_char)
        
        # 將 ID 轉換回字符並返回
        return ''.join([id_to_char[ch_id] for ch_id in char_list])


In [59]:
torch.manual_seed(2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda': print("GPU is ready")
else: print("CPU is ready")

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

GPU is ready


In [60]:
import torch.nn.functional as F

# 定義損失函數，使用交叉熵損失，並忽略 <pad> 標記
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 定義優化器，使用 Adam 或 AdamW
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 或者使用 AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [62]:
from tqdm import tqdm
from copy import deepcopy

model = model.to(device)
best_em = 0.0
i = 0

for epoch in range(1, epochs+1):
    # Training phase
    model.train()
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        optimizer.zero_grad()
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        max_length = max(batch_y.shape[1], batch_pred_y.shape[1])

        if batch_y.shape[1] < max_length:
            pad_size = max_length - batch_y.shape[1]
            pad_tensor = torch.full((batch_y.shape[0], pad_size), fill_value=char_to_id['<pad>']).to(device)
            batch_y = torch.cat([batch_y.to(device), pad_tensor], dim=1)

        if batch_pred_y.shape[1] < max_length:
            pad_size = max_length - batch_pred_y.shape[1]
            pad_tensor = torch.zeros((batch_pred_y.shape[0], pad_size, vocab_size)).to(device)
            batch_pred_y = torch.cat([batch_pred_y, pad_tensor], dim=1)

        pred_y_reshaped = batch_pred_y.reshape(-1, vocab_size)
        y_reshaped = batch_y.reshape(-1).to(device)

        loss = criterion(pred_y_reshaped, y_reshaped)
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), 0.5)
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss=loss.item())

    # Validation phase
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    model.eval()

    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        with torch.no_grad():
            predictions = model(batch_x, batch_x_lens)

        predicted_ids = torch.argmax(predictions, dim=-1)

        for pred, true in zip(predicted_ids, batch_y):
            min_length = min(len(pred), len(true))
            pred = pred[:min_length]
            true = true[:min_length]

            for j in range(min_length):
                if true[j].cpu().item() == char_to_id['<pad>']:
                    pred[j] = char_to_id['<pad>']

            true_length = (true != char_to_id['<pad>']).sum().item()
            if true_length == 0:
                continue

            pred_length = min(len(pred), true_length)
            if np.array_equal(pred[:pred_length].cpu().numpy(), true[:pred_length].cpu().numpy()):
                matched += 1

        total += len(batch_y)

    em = matched / total if total > 0 else 0
    print(f"Exact Match (EM) at epoch {epoch}: {em}")

    # Save the best model
    if em > best_em:
        best_em = em
        best_model = deepcopy(model)


Validation epoch 1: 100%|██████████| 4114/4114 [03:33<00:00, 19.23it/s]


Exact Match (EM) at epoch 1: 0.9553010446343779


Validation epoch 2: 100%|██████████| 4114/4114 [03:24<00:00, 20.16it/s]


Exact Match (EM) at epoch 2: 0.9551377018043685


Validation epoch 3: 100%|██████████| 4114/4114 [04:10<00:00, 16.43it/s]


...

Exact Match (EM) at epoch 8: 0.9550123456790124


Validation epoch 9: 100%|██████████| 4114/4114 [03:37<00:00, 18.91it/s]


Exact Match (EM) at epoch 9: 0.9550009496676163


Validation epoch 10: 100%|██████████| 4114/4114 [03:36<00:00, 18.98it/s]

Exact Match (EM) at epoch 10: 0.9550237416904084


# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [51]:
model = model.to("cpu")
print("".join(model.generator('1+1=')))

1+1=++++++++++++++++++++++++++++++++++++++++++++++
